# LinkedIn Profile Extraction and Validation Workflow

This notebook walks through a workflow for extracting structured data from LinkedIn profiles using an LLM, validating the outputs, and preparing the data for domain expert review. The key steps include:
1. Extracting structured JSON data from unstructured LinkedIn text.
2. Validating the JSON output.
3. Flagging suspicious profiles based on missing or invalid fields.
4. Saving the data to a CSV file.
5. Reviewing and annotating the profiles in an interactive table.

This process can be applied to LinkedIn profiles for individuals or companies to assist with data validation, annotation, and review.

## Step 1: Testing the Pipeline with One Profile

Before applying the process to all profiles, we start by testing the pipeline with a single LinkedIn profile. This ensures that:
- The LLM generates structured JSON data.
- The JSON output is valid and complete.

We'll use one text file (`<profile_name>.txt`) as input and observe the output. 

In [3]:
# Read the file content into a string
file_path = "data/hbaLI.txt"  # Replace with your actual file path

# Open the file in read mode and read the content
with open(file_path, "r", encoding="utf-8") as file:
    linkedin_text = file.read()

# Print the content to verify
# print(linkedin_text)

In [15]:
! pip install --upgrade openai

  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [ ]:
import os
from openai import OpenAI


## os.environ["OPENAI_API_KEY"] = 
client = OpenAI()


# Define the user message (chat-style format)
messages = [
    {"role": "user", "content": f"""
Extract the following structured information from the text below:
- Name
- Current Role
- Location
- Previous Roles
- Education

Text: {linkedin_text}

Output the result as a JSON object.
"""}
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)
output = response.choices[0].message.content
print(output)


TypeError: Client.__init__() got an unexpected keyword argument 'proxies'

In [ ]:
import json

def test_valid_json(output):
    try:
        parsed_output = json.loads(output)
        assert isinstance(parsed_output, dict), "Output is not a valid JSON object"
        print("Valid JSON!")
    except json.JSONDecodeError as e:
        print(f"Invalid JSON: {e}")
        raise

# Test the output
test_valid_json(output)

Invalid JSON: Expecting value: line 1 column 1 (char 0)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Step 2: Switching to OpenAI JSON Mode

Using OpenAI's JSON mode ensures that the output is structured as valid JSON by design. This step improved reliability and reduced the need for custom validation logic. We:
- Re-ran the API call with `response_format={"type": "json_object"}`.
- Validated that the output adheres to the required JSON structure.

Below is the implementation of JSON mode with a single test profile.

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    response_format={ "type": "json_object"},
    messages=messages
)
output = response.choices[0].message.content
print(output)

In [ ]:
# Test the output
test_valid_json(output)

In [ ]:
def test_structure(parsed_output):
    required_fields = ["Name", "Current Role", "Location", "Previous Roles", "Education"]
    assert all(field in parsed_output for field in required_fields), "Missing required fields"
    assert isinstance(parsed_output["Previous Roles"], list), "Previous Roles should be a list"
    assert isinstance(parsed_output["Education"], list), "Education should be a list"
    print("Structure test passed!")

# Run the structure test
parsed_output = json.loads(output)
print(parsed_output)
test_structure(parsed_output)

## Step 3: Batch Processing LinkedIn Profiles

After validating JSON mode with one profile, we extend the workflow to process multiple profiles. Text files containing LinkedIn profile data are stored in a `data/` directory and processed in batch. Key steps include:
- Reading each text file as input.
- Extracting structured data using OpenAI's API.
- Validating the JSON output for each profile.

In [ ]:
# Directory where LinkedIn profile files are stored
profile_dir = "data"

# Function to validate JSON
def validate_json(raw_output):
    try:
        parsed_output = json.loads(raw_output)
        return parsed_output, True
    except json.JSONDecodeError as e:
        print(f"Invalid JSON: {e}")
        return None, False

# Function to process profiles
def process_profiles(directory):
    profiles = []
    for file in os.listdir(directory):
        if file.endswith(".txt"):
            with open(os.path.join(directory, file), "r") as f:
                linkedin_text = f.read()
                print(f"Processing: {file}")
                
                # Define the prompt for the LLM
                messages = [
                    {"role": "user", "content": f"""
                    Extract the following structured information from the text below:
                    - Name
                    - Current Role
                    - Location
                    - Previous Roles
                    - Education

                    Text: {linkedin_text}

                    Output the result as a JSON object.
                    """}
                ]

                # Make LLM API call
                try:
                    response = client.chat.completions.create(
                        model="gpt-4o-mini",
                        response_format={ "type": "json_object"},
                        messages=messages
                    )
                    raw_output = response.choices[0].message.content

                    # Validate JSON
                    parsed_output, is_valid = validate_json(raw_output)
                    if is_valid:
                        profiles.append({"file_name": file, "parsed_output": parsed_output})
                    else:
                        print(f"Skipping invalid JSON for file: {file}")
                except Exception as e:
                    print(f"Error processing file {file}: {e}")
    return profiles

In [ ]:
profiles = process_profiles("data")

## Step 4: Validating JSON Output

Each JSON output from the LLM is validated to ensure:
- The JSON is syntactically valid (using `json.loads()`).
- All required fields are present (`Name`, `Current Role`, `Location`, `Previous Roles`, `Education`).
- The data types of fields match expectations:
  - `Previous Roles` and `Education` should be lists.
  - Other fields should be strings.

Profiles failing validation are flagged for review.

In [ ]:
# Function to verify fields and structure
def verify_fields(json_obj):
    required_fields = ["Name", "Current Role", "Location", "Previous Roles", "Education"]
    
    # Check for required fields
    missing_fields = [field for field in required_fields if field not in json_obj]
    if missing_fields:
        print(f"❌ Missing fields: {missing_fields}")
        return False
    else:
        print(f"✅ All required fields are present")
    
    # Check data types for specific fields
    if not isinstance(json_obj["Previous Roles"], list):
        print("❌ 'Previous Roles' should be a list")
        return False
    if not isinstance(json_obj["Education"], list):
        print("❌ 'Education' should be a list")
        return False
    
    # All checks passed
    print("✅ Structure is valid")
    return True

# Apply field verification to profiles
valid_profiles = []
invalid_profiles = []

for profile in profiles:
    file_name = profile["file_name"]
    json_obj = profile["parsed_output"]
    print(f"📝 Verifying: {file_name}")
    
    if verify_fields(json_obj):
        valid_profiles.append(profile)
        print(f"✅ Profile from {file_name} is valid")
    else:
        print(f"❌ Invalid JSON structure in: {file_name}")
        invalid_profiles.append(profile)

# Log the results
print(f"✨ Valid profiles: {len(valid_profiles)}")
print(f"🚨 Invalid profiles: {len(invalid_profiles)}")

In [ ]:
profiles


In [ ]:
profiles[3]

## Step 5: Adding Flags to Suspicious Profiles

Profiles with missing or unexpected fields are flagged for domain expert review. Examples of flags include:
- Missing fields like `Previous Roles` or `Education`.
- Incorrectly formatted fields.

Flags help identify potential issues in profile extraction and validation logic. Below are examples of flagged profiles.

In [ ]:
def flag_suspicious_profiles(profile):
    parsed_output = profile["parsed_output"]
    flags = []

    # Check for empty important fields
    if not parsed_output.get("Previous Roles"):
        flags.append("Empty 'Previous Roles'")
    if not parsed_output.get("Education"):
        flags.append("Empty 'Education'")
    
    # Heuristic for mismatch: Current Role but no Previous Roles/Education
    if parsed_output.get("Current Role") and not parsed_output.get("Previous Roles") and not parsed_output.get("Education"):
        flags.append("No career or educational history provided")
    
    # Additional checks (e.g., raw text clues)
    raw_text = profile.get("raw_text", "")
    if "mission" in raw_text.lower() or "headquarters" in raw_text.lower():
        flags.append("Profile might be a company, not a person")
    
    return flags

# Apply the flagging to profiles
for profile in profiles:
    file_name = profile["file_name"]
    flags = flag_suspicious_profiles(profile)
    if flags:
        print(f"🚩 Flags for {file_name}: {', '.join(flags)}")
    else:
        print(f"✅ No flags for {file_name}")

## Step 7: Saving Data to CSV

Profiles are saved to a CSV file for further analysis and review. The CSV includes:
- Profile name.
- Extracted fields (e.g., `Name`, `Current Role`).
- Flags for validation issues.

In [ ]:
import csv

def save_profiles_with_flags_to_csv(profiles, output_file="profiles_with_flags.csv"):
    with open(output_file, mode="w", newline="", encoding="utf-8") as csvfile:
        # Define the field names for the CSV (adjust as necessary)
        fieldnames = ["file_name", "flags", "Name", "Current Role", "Location", "Previous Roles", "Education"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for profile in profiles:
            flags = flag_suspicious_profiles(profile)
            parsed_output = profile["parsed_output"]

            # Flatten the parsed JSON fields for easier CSV writing
            writer.writerow({
                "file_name": profile["file_name"],
                "flags": ", ".join(flags),
                "Name": parsed_output.get("Name", ""),
                "Current Role": parsed_output.get("Current Role", ""),
                "Location": parsed_output.get("Location", ""),
                "Previous Roles": "; ".join([f"{role.get('Title', '')} at {role.get('Company', '')}" for role in parsed_output.get("Previous Roles", [])]),
                "Education": "; ".join([f"{edu.get('Degree', '')} in {edu.get('Field', '')} from {edu.get('Institution', '')}" for edu in parsed_output.get("Education", [])]),
            })

# Save all profiles with flags to a CSV

In [ ]:
import csv

def save_json_to_csv(profiles, output_file="profiles.csv"):
    # Collect all unique keys from the JSON objects to form the CSV headers
    all_keys = set()
    for profile in profiles:
        parsed_output = profile["parsed_output"]
        all_keys.update(parsed_output.keys())

    # Convert the set of keys into a sorted list to ensure consistent ordering in the CSV
    headers = sorted(list(all_keys))
    headers.insert(0, "file_name")  # Add file_name as the first column
    headers.append("flags")  # Add flags as the last column

    with open(output_file, mode="w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()

        for profile in profiles:
            # Start with the file_name and flags
            row = {"file_name": profile["file_name"], "flags": ", ".join(flag_suspicious_profiles(profile))}
            # Add the parsed JSON fields
            parsed_output = profile["parsed_output"]
            for key in headers:
                if key in parsed_output:
                    value = parsed_output[key]
                    # Convert lists or dictionaries into strings
                    if isinstance(value, list):
                        row[key] = "; ".join([str(item) for item in value])
                    elif isinstance(value, dict):
                        row[key] = str(value)
                    else:
                        row[key] = value
                elif key not in ["file_name", "flags"]:  # Avoid overwriting file_name or flags
                    row[key] = ""  # Leave empty if the field does not exist in the JSON
            writer.writerow(row)

    print(f"🚀 Profiles saved to {output_file}")

# Save to CSV
save_json_to_csv(profiles)

In [ ]:
import pandas as pd
from IPython.display import display, HTML

# Path to your CSV file
csv_file_path = "profiles_with_flags.csv"

# Load the CSV into a DataFrame
try:
    df = pd.read_csv(csv_file_path)

    # Display the DataFrame as a scrollable table
    display(HTML(df.to_html(index=False, notebook=True)))
except FileNotFoundError:
    print(f"❌ The file {csv_file_path} was not found. Please check the path.")
except Exception as e:
    print(f"⚠️ An error occurred: {e}")

## Step 8: Displaying Data as Editable Table

The CSV data is rendered as an interactive table where:
- Flags and extracted fields can be manually reviewed.
- Annotations (e.g., comments, notes) can be added for each profile.

Below is the editable table for reviewing the profiles.

In [ ]:
! pip install dash dash-table pandas

In [ ]:
import dash
from dash import Dash, dash_table, html, Input, Output, ctx
import pandas as pd

# Load your CSV into a DataFrame
csv_file_path = "profiles_with_flags.csv"
df = pd.read_csv(csv_file_path)

# Create a Dash app
app = Dash(__name__)

# Layout of the app
app.layout = html.Div([
    html.H1("Editable Profiles Table"),
    dash_table.DataTable(
        id='editable-table',
        columns=[{"name": col, "id": col, "editable": True} for col in df.columns],
        data=df.to_dict('records'),
        editable=True,
        row_deletable=True,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'padding': '5px'},
    ),
    html.Button("Save Changes", id='save-button', n_clicks=0),
    html.Div(id='save-output', style={"margin-top": "20px"})
])

# Callback to save changes back to CSV
@app.callback(
    Output('save-output', 'children'),
    Input('save-button', 'n_clicks'),
    Input('editable-table', 'data')
)
def save_to_csv(n_clicks, rows):
    if "save-button" == ctx.triggered_id:  # Ensure save button was clicked
        edited_df = pd.DataFrame(rows)
        edited_df.to_csv(csv_file_path, index=False)
        return "✅ Changes saved to 'profiles_with_flags.csv'"
    return ""

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)